# Setup Environment and Imports
Import required libraries including pathlib, xarray, and the ProcessFiles class from process_files.py

In [1]:
# Import required libraries
import pathlib
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
#go to parent folder
import os
os.chdir('..')
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

# Load and Configure ProcessFiles Class
Initialize the ProcessFiles class with appropriate parameters and configuration settings

In [50]:
#Load subocean data 
SUBOCEAN_PATH = r"data\subocean\forel\CH4\L2B_forel_SubOceanExperiment2024-07-03T12-47-08.nc"
CTD_PATH = r"data\ctd\forel\seabird\2024-07-03T050545 SBE0251267.csv"

subocean_ds = xr.open_dataset(SUBOCEAN_PATH)
ctd_ds = pd.read_csv(CTD_PATH).to_xarray()
_, index = np.unique(subocean_ds.datetime, return_index=True)
subocean_ds = subocean_ds.isel(datetime=index)


In [109]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with two x-axes
fig = make_subplots(
    specs=[[{"secondary_y": False}]]
)

var1 = "_CH4_dissolved_with_constant_dry_gas_flow_ppm_"
var2 = "temperature_seabird"

depth_min = min(subocean_ds.Depth_meter_.min(), ctd_ds.depth_seabird.min())
depth_max = max(subocean_ds.Depth_meter_.max(), ctd_ds.depth_seabird.max())

ch4_min = subocean_ds[var1].min()
ch4_max = subocean_ds[var1].max()
temp_min = ctd_ds[var2].min()
temp_max = ctd_ds[var2].max()

# Add SubOcean CH4 trace
fig.add_trace(
    go.Scatter(
        x=subocean_ds[var1],
        y=subocean_ds.Depth_meter_,
        name="SubOcean CH4",
        mode='markers',
        hovertemplate="CH4: %{x:.2f} ppm<br>Depth: %{y:.1f}m<extra></extra>"
    ),
)

# Add CTD temperature trace — use a separate x-axis
fig.add_trace(
    go.Scatter(
        x=ctd_ds[var2],
        y=ctd_ds.depth_seabird,
        name="CTD Temperature",
        mode='markers',
        hovertemplate="Temp: %{x:.2f}°C<br>Depth: %{y:.1f}m<extra></extra>",
        xaxis='x2',  # Second x-axis
    ),
)

# Update layout
fig.update_layout(
    title="CH4 and Temperature Profiles",
    height=800,
    width=800,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ),
    xaxis=dict(
        title="CH4 (ppm)",
        side="bottom",
        range=[ch4_min, ch4_max]
    ),
    xaxis2=dict(
        title="Temperature (°C)",
        side="top",
        range=[temp_min, temp_max],
        overlaying="x",  # Overlay the axes on the same plot area
        anchor='y'  # Anchor to the same y-axis
    ),
    yaxis=dict(
        title="Depth (m)",
        range=[depth_max, depth_min]
    )
)

fig.show()
